In [22]:
import xgboost as xgb

import numpy as np
import os
import pandas as pd

from sklearn.model_selection import train_test_split
import sklearn

In [23]:
folder_train = '../features_train_in_use'

features_train = pd.read_csv(os.path.join(folder_train, os.listdir(folder_train)[0]))
for i in range(1,len(os.listdir(folder_train))):
    f = os.path.join(folder_train,os.listdir(folder_train)[i])
    if os.path.isfile(f):
        features_train = pd.concat([features_train, pd.read_csv(f)], axis=1)


In [24]:
folder_test = '../features_test_in_use'

features_test = pd.read_csv(os.path.join(folder_test, os.listdir(folder_test)[0]))
for i in range(1,len(os.listdir(folder_test))):
    f = os.path.join(folder_test,os.listdir(folder_test)[i])
    if os.path.isfile(f):
        features_test = pd.concat([features_test, pd.read_csv(f)], axis=1)

In [25]:
target_train = pd.read_csv("../data/stores_train.csv")
target_train = target_train['revenue'].values

#log transform
target_train = np.log1p(target_train)

In [26]:
model = xgb.XGBRegressor(tree_method="gpu_hist", enable_categorical=True)

In [27]:
# TRAIN

for col in features_train.columns:
    if col.startswith("categorical"):
        print(col)
        features_train[col] = features_train[col].astype("category")

categorical_chain_name
categorical_mall_name
categorical_plaace
categorical_sales_channel_name
categorical_street


In [28]:
# TEST

for col in features_test.columns:
    if col.startswith("categorical"):
        print(col)
        features_test[col] = features_test[col].astype("category")

categorical_chain_name
categorical_mall_name
categorical_plaace
categorical_sales_channel_name
categorical_street


In [29]:
model.fit(features_train, target_train)
xg_predictions = model.predict(features_test)

#Undo log transform
xg_predictions = np.expm1(xg_predictions)
print(xg_predictions)

[4.9116154 3.0952258 0.8761778 ... 7.566284  6.901093  2.7704642]


In [30]:
X_train, X_test, y_train, y_test = train_test_split(features_train, target_train, test_size=0.2, random_state=42)

model2 = xgb.XGBRegressor(tree_method="gpu_hist", enable_categorical=True)

#log transform
y_train = np.log1p(y_train)

model2.fit(X_train, y_train)
y_pred = model2.predict(X_test)

y_pred = np.expm1(y_pred)

print("Score: ",sklearn.metrics.mean_squared_log_error(y_test, y_pred) ** 0.5)

Score:  0.3364523000250237


In [31]:
stores_test = pd.read_csv("../data/stores_test.csv")
stores_test_id = stores_test['store_id'].values

predictions = pd.DataFrame()

predictions['id'] = stores_test_id
predictions['predicted'] = xg_predictions

predictions.to_csv("../predictions/xgboost1.csv", index=False)
print(predictions)

                              id  predicted
0     914206820-914239427-717245   4.911615
1     916789157-916823770-824309   3.095226
2       913341082-977479363-2948   0.876178
3      889682582-889697172-28720   0.447799
4     997991699-998006945-417222  11.602345
...                          ...        ...
8572  917323003-917383529-844309   4.781514
8573  917353379-917411824-845904   4.260348
8574  917072302-917089248-833647   7.566284
8575  916960557-916993161-829908   6.901093
8576   987280891-972040746-45320   2.770464

[8577 rows x 2 columns]
